In [1]:
from source.util import utils as u
from source import metrics, plots
from source import handshake, handshake2, scargc, hs
import sys
import time
import os
import psutil
import resource
import numpy as np
from sklearn.metrics import accuracy_score, matthews_corrcoef, f1_score

In [2]:
def doTrain(data, label, step, n_features, percent, epsilon, n_components, k):

    label_ = label
    data = np.vsplit(data, 10)
    label = u.chunkify(label, 10)

    # print(len(data))
    # print(len(label))

    # percent = int(percent)
    # print(percent)
    # print(epsilon)
    acc_percent = []
    f1_percent = []
    mcc_percent = []
    predicted = []
    updt = []

    pred, updt = handshake2.handshake2(data, label_, data[0], label[0], data[1], label[1], n_components, n_features, epsilon, percent, k)
    pred = np.array(pred, dtype=int)
    # print(label[i])
    pred = pred.flatten()
    predicted = pred
    acc_percent, f1_percent, mcc_percent = metrics.makeBatches(label[0], predicted, len(label[0]), step)

    # print(acc_percent, f1_percent, mcc_percent)
    # sys.exit(0)

    for i in range(1, len(data)-1):
        # print(i)
        pred, upd = handshake2.handshake2(data, label_, data[i], label[i], data[i+1], label[i+1], n_components, n_features, epsilon, percent)
        pred = np.array(pred, dtype=int)
        # print(label[i])
        pred = pred.flatten()

        # print(pred)
        acc, f1, mcc = metrics.makeBatches(label[i], pred, len(label[i]), step)

        predicted = np.vstack([predicted, pred])
        # print(predicted)
        # sys.exit(0)

        updt += upd
        acc_percent = np.vstack([acc_percent, acc])
        f1_percent = np.vstack([f1_percent, f1])
        mcc_percent = np.vstack([mcc_percent, mcc])


    # print(acc_percent)

    predicted = predicted.flatten()
    tam = len(label_[0])
    lab = label_[tam:]


    score = accuracy_score(lab, predicted)
    f1 = f1_score(lab, predicted, average = 'macro')
    mcc = matthews_corrcoef(lab, predicted)
    # score, f1, mcc = metrics.metrics(acc_percent, lab, predicted, step, f1_type = 'macro')
    # print(a,score, f1, mcc)
    return score, f1, mcc

In [3]:
poolsize = 300
clusters = 2
n_components = 2
# episilon = 0.15
step = 10

In [4]:
base = '/home/god/Documents/ccomp/tcc/datasets/'
list = ['1CDT.txt', '1CHT.txt']# '1CSurr.txt', '2CDT.txt', '2CHT.txt']# 'NOAA.txt', 'elec.txt', 'keystroke.txt']
# list = ['keystroke.txt']
# list = ['NOAA.txt']#, 'elec.txt']
database = {}

for i in range(0, len(list)):
    database[i] = base + list[i]

array_ep = [0.05, 0.10, 0.15]
# array_ep = [0.1]
array_p = [10, 20, 30]
array_k = [50, 25, 12, 7]
# array_p = [30]

result = [0.0, 0.0, 0.0]
aux = []
result = np.array(result, dtype=float)

dic = {}

In [5]:
dic = {}
for key, value in database.items():
    dic[key] = {}
    for ep in range(0, len(array_ep)):
        for p in range(0, len(array_p)):
            for k in range(0, len(array_k)):
                adr = value
                dataset, data_labeled, dataset_train, l_train, stream, l_stream, n_features = u.criar_datasets(5, adr)

                score, f1, mcc = doTrain(dataset_train, l_train, step, n_features, array_p[p], array_ep[ep], n_components, array_k[k])

                aux = np.hstack([score, f1, mcc])
                result = np.vstack([result, aux])

                a = str(array_ep[ep])
                b = str(array_p[p])
                name = a +' / '+ b + ' / '+ k
                dic[key][name] = aux

/home/god/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/god/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:543: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/god/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/god/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:543: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/god/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-def

/home/god/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/god/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:543: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/god/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/god/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/god/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:543: RuntimeWarning: invalid 

/home/god/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/god/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:543: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/god/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/god/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:543: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/god/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 

In [6]:
for p_id, p_info in dic.items():
    print('dataset :', p_id)

    for key in p_info:
        print(key, ':', p_info[key])

dataset : 0
0.05 / 10 : [0.50416667 0.50395137 0.00876126]
0.05 / 20 : [0.52916667 0.52915849 0.05848238]
0.05 / 30 : [0.52083333 0.52056606 0.04137533]
0.1 / 10 : [0.5        0.49944383 0.00062679]
0.1 / 20 : [ 0.47916667  0.47887615 -0.04126887]
0.1 / 30 : [0.54444444 0.54444093 0.08901631]
0.15 / 10 : [ 0.48472222  0.48449848 -0.03017768]
0.15 / 20 : [0.52916667 0.52909309 0.05864574]
0.15 / 30 : [ 0.48472222  0.48388493 -0.02993434]
dataset : 1
0.05 / 10 : [0.55555556 0.55506118 0.11135885]
0.05 / 20 : [ 0.45555556  0.45504676 -0.08905534]
0.05 / 30 : [ 0.49305556  0.48962847 -0.01407926]
0.1 / 10 : [0.52222222 0.52198617 0.04448841]
0.1 / 20 : [ 0.48055556  0.47925369 -0.0390848 ]
0.1 / 30 : [ 0.45694444  0.45693502 -0.0861141 ]
0.15 / 10 : [ 0.43472222  0.42673253 -0.13435428]
0.15 / 20 : [ 0.43472222  0.43187172 -0.1318857 ]
0.15 / 30 : [ 0.49166667  0.49156859 -0.0166731 ]
